# Convolutional Neural Network

### Packages

In [71]:
# ----- Tensorflow -----
import tensorflow as tf
import tensorflow_datasets as tfds
from keras import regularizers as reg
from keras import optimizers opt
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.vis_utils import plot_model

from keras.layers import Input, Dropout, Dense, Conv2D, MaxPooling2D, Flatten, Concatenate, AveragePooling2D

# ----- Utility functions -----
from utils import load_data, plot_hist

### Load Data

In [56]:
data_dir = '/Users/gustavchristensen/Documents/SDU/MSc. Data Science/3. Semester - DT/Anvendt Maskinlæring'
BATCH_SIZE = 32
train, test, val = load_data(data_dir, perc=1, batch_size=BATCH_SIZE)

### Architectures
- Consider and discuss alternative CNN model architectures

In [1]:
# ----- Small Convnet from DLPR -----
inputs = Input(shape=(28, 28, 1))
x = Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = MaxPooling2D(pool_size=2)(x)
x = Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = MaxPooling2D(pool_size=2)(x)
x = Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = Flatten()(x)
outputs = Dense(2, activation="softmax")(x)
model = Model(inputs=inputs, outputs=outputs)

NameError: name 'Input' is not defined

In [ ]:
# ----- Configure model -----
model.compile(optimizer=opt.Nadam(learning_rate=0.0002),
	loss='categorical_crossentropy',
	metrics=['accuracy'])

# ----- Train model -----
history = model.fit(train, epochs = 10, validation_data= val)

## Alternative Optimizers
- Visualize impact on training/validation losses and accuracies

In [ ]:
# ----- SGD -----
opt1 = tf.keras.optimizers.SGD(learning_rate=0.001)

# ----- RMSProp -----
opt2 = tf.keras.optimizers.RMSprop(learning_rate=0.001)

# ----- Adagrad -----
opt3 = tf.keras.optimizers.Adagrad(learning_rate=0.001)

# ----- Adam -----
opt4 = tf.keras.optimizers.Adam(learning_rate=0.001)


## Data Augmentation
- Visualize impact on training/validation losses and accuracies
- Discuss data augmentation and its relation to overfitting